This notebook serves as a template with the initial steps we took to load the dataset. You can use it as a guideline for your own implementation. Please note that since the dataset is obtained from the NSD experiment, bounded for the Algonauts Project challenge, you
may be asked at some point to provide an email address associated with a drive account to access the data.






# 0.1 Install and import the necessary libraries

In [ ]:
!pip install nilearn==0.9.2

In [ ]:
import os
import numpy as np
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
from nilearn import datasets
from nilearn import plotting
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as corr

# 0.2 Access the data

If you are running the notebooks on Google Colab you can use the Challenge data located in a Google Drive public folder called algonauts_2023_tutorial_data (fill out this [form](https://docs.google.com/forms/d/e/1FAIpQLSehZkqZOUNk18uTjRTuLj7UYmRGz-OkdsU25AyO3Wm6iAb0VA/alreadyresponded) to get the folder link). This folder contains the unzipped data of each subject.

Before running the code you need to select this folder and choose "Add a shortcut to Drive". This will create a shortcut (without copying or taking space) of the folder to a desired path, for instance, in your Google Drive, from which you can read the content after mounting using drive.mount().

Finally, edit the data_dir variable below with the path on your Drive to the algonauts_2023_tutorial_data shortcut folder, and the parent_submission_dir variable with the path on your Drive where you wish to save the predicted test fMRI data.

In [ ]:
 from google.colab import drive
    drive.mount('/content/drive/', force_remount=True)
    data_dir = '/content/drive/MyDrive/algonauts_2023_tutorial_data'
    parent_submission_dir = '/content/drive/MyDrive/algonauts_2023_challenge_submission'

# 0.3 Select GPU

In [ ]:
device = 'cuda' #@param ['cpu', 'cuda'] {allow-input: true}
device = torch.device(device)

# 1 Load the data

In [ ]:
# Choose one of the 8 subjects

subj = 1 #@param ["1", "2", "3", "4", "5", "6", "7", "8"] {type:"raw", allow-input: true}

# 1.1 Define paths
We define some paths in our drive needed for loading and storing the data

In [ ]:
class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj):

    self.subj = format(subj, '02')
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)

    # create the submission directory if not existing
    if not os.path.isdir(self.subject_submission_dir):
        os.makedirs(self.subject_submission_dir)

args = argObj(data_dir, parent_submission_dir, subj)

# 1.2 Load the fMRI training data
The fMRI data consists of two '.npy' files:

    lh_training_fmri.npy: the left hemisphere (LH) fMRI data.
    rh_training_fmri.npy: the right hemisphere (RH) fMRI data.

Both files are 2-dimensional arrays with training stimulus images as rows and fMRI vertices as columns.

In [ ]:
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

print('LH training fMRI data shape:')
print(lh_fmri.shape)
print('(Training stimulus images × LH vertices)')

print('\nRH training fMRI data shape:')
print(rh_fmri.shape)
print('(Training stimulus images × RH vertices)')